In [1]:
# import re
import random
import pickle

from io import open
from tqdm import tqdm

# Torch
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# Word Tokenizere
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

# Sentence similarity evaluation
import spacy
#!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

# Set CUDA as device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
device

device(type='cuda')

#### Load the dataset

In [3]:
with open('/content/drive/MyDrive/InforetProject/Datasets/entails_dataset.pickle', 'rb') as handle:
    entails_dataset = pickle.load(handle)

In [4]:
with open('/content/drive/MyDrive/InforetProject/Datasets/imdb_dataset.pickle', 'rb') as handle:
    imdb_dataset = pickle.load(handle)

#### Create a corpus using the dataset

In [5]:
SOS_token = 0
EOS_token = 1

class Corpus:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in word_tokenize(sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

def indexesFromSentence(corpus, sentence):
    return [corpus.word2index[word] for word in word_tokenize(sentence)]

def tensorFromSentence(corpus, sentence):
    indexes = indexesFromSentence(corpus, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(corpus, pair[0])
    target_tensor = tensorFromSentence(corpus, pair[1])
    return (input_tensor, target_tensor)

In [6]:
corpus = Corpus()
MAX_LENGTH = 20
dataset = []

for pair in entails_dataset:
    corpus.addSentence(pair[0])
    corpus.addSentence(pair[1])
    if len(word_tokenize(pair[0])) < MAX_LENGTH and len(word_tokenize(pair[1])) < MAX_LENGTH:
        dataset.append(pair)

# Increase the number of words in the corpus
for text in imdb_dataset[100]:
    corpus.addSentence(text)

#### Divide training set and test set

In [7]:
random.shuffle(dataset)
trainingSetSize = len(dataset) // 100 * 80

trainingSet = dataset[:trainingSetSize]
testSet = dataset[trainingSetSize:]

#### Define Encoder and Decoder

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

#### Training Algorithm

In [10]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, train=True):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(MAX_LENGTH, 
                                  encoder.hidden_size, 
                                  device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], 
            encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # use_teacher_forcing = False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, 
                decoder_hidden, 
                encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, 
                decoder_hidden, 
                encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    if train:
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

    return loss.item() / target_length

In [11]:
def trainIters(trainingSet, encoder, decoder, testSet=None, n_iters=1000, plot_every=100, learning_rate=0.01, test_ratio=0.2, test=False):
    plot_losses = []
    plot_loss_total = 0

    plot_test_losses = []
    plot_test_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(trainingSet)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in tqdm(range(1, n_iters + 1)):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, 
                     target_tensor, 
                     encoder,
                     decoder, 
                     encoder_optimizer, 
                     decoder_optimizer, 
                     criterion)
        plot_loss_total += loss
        
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

            if test:
                test_set_size = int(n_iters // 100 * (test_ratio*100))
                testing_pairs = [tensorsFromPair(random.choice(testSet)) for i in range(test_set_size)]

                for test_iter in range(1, test_set_size + 1):
                    testing_pair = testing_pairs[test_set_size - 1]
                    input_tensor = testing_pair[0]
                    target_tensor = testing_pair[1]

                    loss = train(input_tensor, 
                                target_tensor, 
                                encoder,
                                decoder, 
                                encoder_optimizer, 
                                decoder_optimizer, 
                                criterion,
                                train=False)
                    plot_test_loss_total += loss

                plot_loss_avg = plot_test_loss_total / test_set_size
                plot_test_losses.append(plot_loss_avg)
                plot_test_loss_total = 0
            
    if test:
        return plot_losses, plot_test_losses
    else:
        return plot_losses

#### Evaluation

In [12]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(corpus, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(
            max_length, 
            encoder.hidden_size, 
            device=device)
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei],
                encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, 
                decoder_hidden, 
                encoder_outputs)
            
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(corpus.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

def evaluateRandomly(encoder, decoder, pairs, n=100, print_output=True, output=False):
    out = []
    for i in range(n):
        pair = random.choice(pairs)
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        if print_output:
            print('>', pair[0], '\n=', pair[1], '\n<', output_sentence, '\n')
        if output:
            out.append((pair[0], pair[1], output_sentence))
    if output: return out

def evaluateSentence(sentence):
    output_words, attentions = evaluate(encoder, attn_decoder, sentence)
    output_sentence = ' '.join(output_words)
    print('>', sentence, '\n<', output_sentence, '\n')

def evaluate_model(pairs, n_samples=100):
    sim = []
    eva = evaluateRandomly(encoder, attn_decoder, pairs, n=n_samples, print_output=False, output=True)
    for e in eva:
        _, target, answer = e
        sim.append(nlp(target).similarity(nlp(answer)))           
    return sum(sim) / len(sim)

#### Model Training

In [13]:
hidden_size = 256

encoder = EncoderRNN(corpus.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, 
                              corpus.n_words, 
                              dropout_p=0.1).to(device)

train_loss = trainIters(trainingSet, encoder, attn_decoder, n_iters=75000, plot_every=7500, test=False)

In [14]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,3))
ax.plot(train_loss, "r", marker='o')
plt.show()

'import matplotlib.pyplot as plt\nfig, ax = plt.subplots(figsize=(12,3))\nax.plot(train_loss, "r", marker=\'o\')\n# ax.plot(test_loss, "g", marker=\'o\')\nplt.show()'

In [16]:
encoderStatePath = "/content/drive/MyDrive/InforetProject/ModelState/textualEntails_encoder.pth"
decoderStatePath = "/content/drive/MyDrive/InforetProject/ModelState/textualEntails_decoder.pth"
# torch.save(encoder.state_dict(), encoderStatePath)
# torch.save(attn_decoder.state_dict(), decoderStatePath)
encoder.load_state_dict(torch.load(encoderStatePath))
attn_decoder.load_state_dict(torch.load(decoderStatePath))

<All keys matched successfully>

In [17]:
train_out = evaluateRandomly(encoder, attn_decoder, trainingSet, 100, output=True, print_output=False)
trainOutput = "/content/drive/MyDrive/InforetProject/trainOutput.txt"
f = open(trainOutput, "a")
for tri in train_out:
    f.write(tri[0]+'\n')
    f.write(tri[1]+'\n')
    f.write(tri[2]+'\n\n')
f.close()

In [18]:
test_out = evaluateRandomly(encoder, attn_decoder, testSet, 100, output=True, print_output=False)
testOutput = "/content/drive/MyDrive/InforetProject/testOutput.txt"
f = open(testOutput, "a")
for tri in test_out:
    f.write(tri[0]+'\n')
    f.write(tri[1]+'\n')
    f.write(tri[2]+'\n\n')
f.close()

In [19]:
evaluateSentence("Food is good enough")

> Food is good enough 
< food is be <EOS> 



In [20]:
evaluateSentence("There is enough leg room")

> There is enough leg room 
< The seats could do with more legroom <EOS> 



In [22]:
evaluateSentence("I would like a fruit")

> I would like a fruit 
< Food food be with more choice <EOS> 



In [23]:
evaluateSentence("There is not enough time")

> There is not enough time 
< food is average <EOS> 



### Model Evaluatioin

Performance on training set

Performance on test set

Ability to generalize 